In [ ]:
import datetime as dt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from google.colab import drive

In [ ]:
drive.mount('drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at drive


In [ ]:
import json
!pip install "ibm-watson>=4.5.0"
from ibm_watson import ToneAnalyzerV3, ApiException
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

     |████████████████████████████████| 378kB 2.7MB/s 
     |████████████████████████████████| 204kB 8.5MB/s 
  Created wheel for ibm-watson: filename=ibm_watson-4.5.0-cp36-none-any.whl size=364301 sha256=30372141cb261d67032366810af808dbd49360ec8aba179aef12d9a270b301d1
  Stored in directory: /root/.cache/pip/wheels/71/9a/0a/9b3ca8eca69bc5362eb04709a750b30055a9d27818fd0c9494
  Created wheel for ibm-cloud-sdk-core: filename=ibm_cloud_sdk_core-1.5.1-cp36-none-any.whl size=44491 sha256=730a5a29ed03292d4d2e1262e7a2273879a869aa34bedd5c7544eea61fc91f2a
  Stored in directory: /root/.cache/pip/wheels/6a/42/50/f96888116b329578304f9dda4693cef6f3e76e18272d22cb6c
Successfully built ibm-watson ibm-cloud-sdk-core


In [ ]:
# language processing libs
!pip install vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import re
!pip install emoji
import emoji

     |████████████████████████████████| 133kB 2.8MB/s 
     |████████████████████████████████| 51kB 1.7MB/s 
  Created wheel for emoji: filename=emoji-0.5.4-cp36-none-any.whl size=42176 sha256=152eed774d65d0a3f1dc1eda5d9e73e0ab4be3754270d8386737dee6ff71c40d
  Stored in directory: /root/.cache/pip/wheels/2a/a9/0a/4f8e8cce8074232aba240caca3fade315bb49fac68808d1a9c
Successfully built emoji


##Data Preprocessing

In [ ]:
def data_pre_processing(data_frame_arg):
  # data preprocessing
  # removing unwanted stuff from text of tweet using regex

  #replacing & with and
  data_frame_arg["clean_text"] = data_frame_arg["text"].to_numpy()

  #Removing URLs
  def removeURLs(str):
      return re.sub(r'https?://\S+', ' ', str)
  data_frame_arg["clean_text"] = data_frame_arg["clean_text"].apply(lambda tweet: removeURLs(tweet))

  def proc(text):
    text = emoji.demojize(text, use_aliases=True, delimiters=('','')).replace('_', ' ').replace('&', 'and').replace('.', '. ').replace(',', ', ').replace('#', '# ').replace('?', '? ').replace('!', '! ').replace('&', 'and')
    return re.sub(r'\s+', ' ', text)
  data_frame_arg["clean_text"] = data_frame_arg["clean_text"].apply(lambda tweet: proc(tweet))

##VADER Sentiment Analysis

In [ ]:
def get_score_df(data_frame_arg):
  sia = SentimentIntensityAnalyzer()
  scores_ = data_frame_arg["text"].apply(lambda tweet: sia.polarity_scores(tweet))
  scores_df_ret = pd.DataFrame(list(scores_))
  scores_df_ret['result'] = scores_df_ret['compound'].apply(lambda res: 'neutral' if res == 0 else ('positive' if res > 0 else 'negative'))
  for col in scores_df_ret:
    data_frame_arg[col] = scores_df_ret[col].to_numpy()

## IBM Watson Tone Analyser





In [ ]:
# IBM Watson Tone Analyser
def get_tones(data_frame_arg):
  row_list = []
  def get_single_tone(text):
    tone_analysis = tone_analyzer.tone({'text': text}, content_type='application/json', sentences=False).get_result()
    dict1 = {'analytical': 0, 'anger': 0,'confident': 0, 'fear': 0, 'joy': 0, 'sadness': 0, 'tentative': 0}
    for tone in tone_analysis['document_tone']['tones']:
      dict1.update({tone['tone_id'] : tone['score']})
    row_list.append(dict1)
  data_frame_arg['clean_text'].apply(lambda tweet: get_single_tone(tweet))
  df = pd.DataFrame(row_list)
  for col in df:
    data_frame_arg[col] = df[col].to_numpy()

In [ ]:
df = pd.read_csv('drive/My Drive/colab_drive/ibm_hackathon_2020/final_lockdown_tweets/MarchFinal.csv')
df.head()

,user,text,date,favorites,retweets,mentions,hashtags
0,Praveen_bhushan,First Made-in-India COVID-19 test kit by Mylab...,2020-03-23 23:59:55+00:00,2,2,@BT_India,#Covid19India #MakeInIndia
1,shekhar_mande,Scaling up Diagnosis in one of the immediate p...,2020-03-23 23:59:33+00:00,26,85,@ccmb_csir @CSIR_IND @3RakeshMishra,#Covid19India
2,MP2139292721,👏🏻👏🏻👏🏻 Indian warriors.. When this is all over...,2020-03-23 23:58:38+00:00,0,0,@BBCWorld @TheSun,#Corona #Coronaindia
3,ArjunChandrakr,COVID-19 has taken THANOS words very seriously...,2020-03-23 23:58:13+00:00,0,0,NaN,#Covid19India #Endgame
4,TPE_connect,"Delhi's Tihar jail to release 3,000 prisoners ...",2020-03-23 23:57:00+00:00,1,0,NaN,#Delhi #India #Lockdown #CoronavirusIndia #COV...


In [ ]:
df2 = df[2500:].copy()
data_pre_processing(df2)
get_score_df(df2)

apikey = '**********'
authenticator = IAMAuthenticator(apikey)
tone_analyzer = ToneAnalyzerV3(
    version='2017-09-21',
    authenticator=authenticator
)
url = '*******************'
tone_analyzer.set_service_url(url)
get_tones(df2)
df2.to_csv('drive/My Drive/colab_drive/ibm_hackathon_2020/final_temp_tweets/df_march2.csv', index=False)
df2.head()

,user,text,date,favorites,retweets,mentions,hashtags,clean_text,neg,neu,pos,compound,result,analytical,anger,confident,fear,joy,sadness,tentative
2500,praveen_dhigan,"Why Delhi government is not providing masks, s...",2020-03-28 23:59:35+00:00,1,0,NaN,#COVID2019india #Delhi #DelhiFightsCorona #Arv...,"Why Delhi government is not providing masks, s...",0.000,1.000,0.000,0.0000,neutral,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000
2501,saifytweetz,"Funds by Indian govt for ""Development Activiti...",2020-03-28 23:59:01+00:00,1,1,NaN,#COVID2019india #PMCARES,"Funds by Indian govt for ""Development Activiti...",0.000,0.935,0.065,0.3182,positive,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000
2502,samar4plant,That's why I too say he is the most incompeten...,2020-03-28 23:58:49+00:00,0,0,NaN,#Coronaindia,That's why I too say he is the most incompeten...,0.127,0.828,0.045,-0.6794,negative,0.566171,0.636235,0.000000,0.0,0.000000,0.0,0.865388
2503,leaderrajs,All Religions waiting for a positive response ...,2020-03-28 23:58:46+00:00,2,3,NaN,#COVID2019india #DoctorLife #scientist #PMOfIn...,All Religions waiting for a positive response ...,0.000,0.806,0.194,0.5574,positive,0.694442,0.000000,0.986725,0.0,0.859894,0.0,0.000000
2504,Aanandshirali,TATA'S SHOULD PLAN TO FIGHT THIS EMERGENCY. YO...,2020-03-28 23:58:29+00:00,0,0,NaN,#COVID19INDIA,TATA'S SHOULD PLAN TO FIGHT THIS EMERGENCY. YO...,0.186,0.595,0.220,0.2846,positive,0.000000,0.612760,0.000000,0.0,0.000000,0.0,0.000000


In [ ]:
df_march_fin = df1.copy()

In [ ]:
df_march_fin = df_march_fin.append(df2, ignore_index=True)
df_march_fin.head()

,user,text,date,favorites,retweets,mentions,hashtags,clean_text,neg,neu,pos,compound,result,analytical,anger,confident,fear,joy,sadness,tentative
0,Praveen_bhushan,First Made-in-India COVID-19 test kit by Mylab...,2020-03-23 23:59:55+00:00,2,2,@BT_India,#Covid19India #MakeInIndia,First Made-in-India COVID-19 test kit by Mylab...,0.000,0.819,0.181,0.4767,positive,0.0,0.0,0.000000,0.000000,0.537332,0.000000,0.0
1,shekhar_mande,Scaling up Diagnosis in one of the immediate p...,2020-03-23 23:59:33+00:00,26,85,@ccmb_csir @CSIR_IND @3RakeshMishra,#Covid19India,Scaling up Diagnosis in one of the immediate p...,0.085,0.787,0.128,0.3182,positive,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0
2,MP2139292721,👏🏻👏🏻👏🏻 Indian warriors.. When this is all over...,2020-03-23 23:58:38+00:00,0,0,@BBCWorld @TheSun,#Corona #Coronaindia,clapping hands light skin toneclapping hands l...,0.000,0.909,0.091,0.5574,positive,0.0,0.0,0.945261,0.000000,0.585211,0.000000,0.0
3,ArjunChandrakr,COVID-19 has taken THANOS words very seriously...,2020-03-23 23:58:13+00:00,0,0,NaN,#Covid19India #Endgame,COVID-19 has taken THANOS words very seriously...,0.316,0.684,0.000,-0.6240,negative,0.0,0.0,0.543112,0.000000,0.000000,0.847391,0.0
4,TPE_connect,"Delhi's Tihar jail to release 3,000 prisoners ...",2020-03-23 23:57:00+00:00,1,0,NaN,#Delhi #India #Lockdown #CoronavirusIndia #COV...,"Delhi's Tihar jail to release 3, 000 prisoners...",0.234,0.766,0.000,-0.7269,negative,0.0,0.0,0.000000,0.539543,0.000000,0.000000,0.0


In [ ]:
df_march_fin.to_csv('drive/My Drive/colab_drive/ibm_hackathon_2020/final_lockdown_tweets/MarchFinalWithSentiments.csv', index=False)